# 텐서플로 2 시작 - 초보용

Keras를 사용하여 다음을 수행합니다.

1. 사전에 빌드한 데이터세트를 로드합니다.
2. 이미지를 분류하는 신경망 머신 러닝 모델을 빌드합니다.
3. 이 신경망을 훈련합니다.
4. 모델의 정확도를 평가합니다.

## Tensorflow 설정

Tensorflow 프로그램 가져옴

In [2]:
import tensorflow as tf

f'Tensorflow version : {tf.__version__}'

'Tensorflow version : 2.10.0'

## 데이터세트 로드

MNIST 데이테세트. 이미지의 픽셀 값 범위는 0~255. 이때 값을 255.0으로 나누어 0~1 범위로 조정

In [3]:
# MNIST 데이터세트
mnist = tf.keras.datasets.mnist

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## 머신 러닝 모델 빌드

tf.keras.Sequential 모델을 빌드

In [10]:
# Sequential은 각 레이어에 하나의 입력 텐서와 하나의 출력 텐서가 있는 레이어를 쌓는 데 유용
# 이 모델은 Flatten, Dense, Dropout 레이어를 사용

In [19]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Flatten(input_shape=(28, 28)), 			# 입력 레이어
	tf.keras.layers.Dense(128, activation='relu'),			# 은닉 레이어
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Dense(10, activation='softmax')
])

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
predictions = model(X_train[:1]).numpy()
predictions

array([[0.10432903, 0.1142545 , 0.12578824, 0.04959985, 0.07319808,
        0.07879159, 0.07749727, 0.09213316, 0.06115898, 0.22324929]],
      dtype=float32)

In [22]:
# tf.nn.softmax 함수는 이러한 로짓을 각 클래스에 대한 확률로 변환
tf.nn.softmax(predictions).numpy()

array([[0.10032129, 0.10132198, 0.10249736, 0.09497832, 0.0972463 ,
        0.09779177, 0.09766528, 0.09910522, 0.09608256, 0.11298987]],
      dtype=float32)

In [23]:
# losses.SparseCategoricalCrossentropy를 사용하여 훈련용 손실 함수를 정의
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [24]:
loss_fn(y_train[:1], predictions).numpy()

2.324915

In [25]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 모델 훈련 및 평가

In [26]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


c:\Users\perso\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 [==============================] - 12s 6ms/step - loss: 0.2912 - accuracy: 0.9167
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.1421 - accuracy: 0.9586
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1062 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0884 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0753 - accuracy: 0.9754


In [27]:
model.evaluate(X_test,  y_test, verbose=2)

c:\Users\perso\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 - 2s - loss: 0.0730 - accuracy: 0.9778 - 2s/epoch - 6ms/step


[0.07300934195518494, 0.9778000116348267]

In [28]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [30]:
probability_model(X_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0853425 , 0.08534237, 0.08534317, 0.0853736 , 0.08534235,
        0.08534237, 0.08534235, 0.23188062, 0.0853425 , 0.08534816],
       [0.08533692, 0.08533744, 0.23196654, 0.08533736, 0.08533692,
        0.08533693, 0.08533692, 0.08533692, 0.08533712, 0.08533692],
       [0.08534072, 0.23190624, 0.08534271, 0.08534084, 0.08534228,
        0.08534089, 0.08534091, 0.08535917, 0.08534544, 0.08534074],
       [0.23192276, 0.08533968, 0.08534929, 0.08533983, 0.08533969,
        0.08534286, 0.08534641, 0.08533981, 0.08533972, 0.08533994],
       [0.08537843, 0.0853776 , 0.08537785, 0.0853776 , 0.23132145,
        0.08537763, 0.08537904, 0.08537807, 0.08537814, 0.08565414]],
      dtype=float32)>

### 결론

축하합니다! Keras API를 사용하는 사전에 빌드한 데이터세트를 사용하여 머신 러닝 모델을 훈련했습니다.